<a href="https://colab.research.google.com/github/devdoe153/llm_voice/blob/main/gpt4_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai python-dotenv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp /content/drive/MyDrive/.env ./

cp: cannot stat '/content/drive/MyDrive/.env': No such file or directory


In [7]:
import os

from dotenv import load_dotenv
# 환경변수 블러오기
from dotenv import load_dotenv, dotenv_values
from openai import OpenAI

config = dotenv_values(".env")
os.environ.update(config)



load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [10]:
from IPython.display import Audio

file_name = "conversation_30s.mp3"

Audio(file_name)

In [17]:
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
model = "whisper-1"

audio_file= open(file_name, "rb")
transcript = client.audio.transcriptions.create(
  model=model,
  file=audio_file,
  response_format="verbose_json",
  timestamp_granularities=["segment", "word"]
)

In [20]:
print(transcript)

TranscriptionVerbose(duration=27.190000534057617, language='korean', text='안녕하세요. 오늘 날씨가 정말 좋네요. 네, 정말 그러네요. 이런 날엔 나가서 운동하기 딱이죠. 운동 좋아하시나 봐요? 요즘 어떤 운동하세요? 저는 주로 조깅을 합니다. 요즘은 주말마다 산책도 하고요. 좋네요. 저도 조깅 해봐야겠어요. 좋은 하루 되세요. 네, 즐거운 하루 되세요.', segments=[TranscriptionSegment(id=0, avg_logprob=-0.23772837221622467, compression_ratio=1.576923131942749, end=3.380000114440918, no_speech_prob=0.001149520045146346, seek=0, start=0.0, temperature=0.0, text=' 안녕하세요. 오늘 날씨가 정말 좋네요.', tokens=[50364, 19289, 13, 8880, 16316, 25416, 1453, 12793, 5008, 35969, 13, 50564]), TranscriptionSegment(id=1, avg_logprob=-0.23772837221622467, compression_ratio=1.576923131942749, end=9.119999885559082, no_speech_prob=0.001149520045146346, seek=0, start=4.199999809265137, temperature=0.0, text=' 네, 정말 그러네요. 이런 날엔 나가서 운동하기 딱이죠.', tokens=[50564, 8808, 11, 12793, 13725, 12974, 13, 8381, 16316, 29840, 37011, 2393, 33541, 22700, 16681, 27036, 13, 50849]), TranscriptionSegment(id=2, avg_logprob=-0.23772837221622467, compression

In [18]:
transcription_text = ""

for segment in transcript.segments:
  transcription_text += segment.text + "\n"


print(transcript.text)

안녕하세요. 오늘 날씨가 정말 좋네요. 네, 정말 그러네요. 이런 날엔 나가서 운동하기 딱이죠. 운동 좋아하시나 봐요? 요즘 어떤 운동하세요? 저는 주로 조깅을 합니다. 요즘은 주말마다 산책도 하고요. 좋네요. 저도 조깅 해봐야겠어요. 좋은 하루 되세요. 네, 즐거운 하루 되세요.


In [21]:
prompt = f"다음 회의록을 요약해주세요\n\n{transcription_text}"

summary = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ]
).choices[0].message.content

print(summary)

- 날씨가 좋다는 이야기로 시작
- 운동에 대한 대화
- 조깅을 주로 하는 사람
- 산책도 하면서 휴식
- 조깅을 시도하겠다는 의지 표현
- 서로 즐거운 하루를 기원함
- 회의는 운동에 대한 얘기로 끝나며 각자 일상으로 돌아감.


In [22]:
prompt = f"이 글에서 중심 주제아 주요 결론을 포함하 요약해주세요\n\n{transcription_text}"

summary = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ]
).choices[0].message.content

print(summary)

이 대화의 주제는 운동과 날씨에 관한 것입니다. 주요 결론은 날씨가 좋을 때는 운동하기 좋고, 조깅이나 산책을 하면 좋다는 것입니다. 함께 운동을 즐기는 대화였습니다.


In [23]:
prompt = """다음 글의 주제, 주요 주장, 결론을 포함하여 3문장으로 요약해주세요. 각 포인트를 명확하게 구분하여 json형식으로 출력해주세요"""
prompt += f"\n\n{transcription_text}"

summary = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ]
).choices[0].message.content

print(summary)

{
  "주제": "대화 속에서의 운동에 대한 이야기",
  "주요 주장": "날씨가 좋은 날엔 운동하기 좋다는 주장이 나옴",
  "결론": "운동을 즐겁게 하는 것이 중요하다는 결론이 도출됨"
}
